In [8]:
from __future__ import print_function
import pandas as pd
from sklearn.cross_validation import train_test_split
import tensorflow as tf
learn = tf.contrib.learn

In [9]:
def load_clean_data(csv_file):
    df = pd.read_csv(csv_file, header = 0)
    # Normalize "Age"
    df['Age'] = df['Age'].fillna(df.Age.median())
    df['Age'] = (df['Age'] - 40)/80
    # One hot encoding for "Sex"
    df['Gender'] = df['Sex'].map({'female':0, 'male':1}).astype(int)
    # Normalize "Fare"
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df.loc[df['Fare'] > 100,'Fare'] = 100
    df['Fare'] = (df['Fare'] - 50)/100
    df['Embarked'] = df['Embarked'].fillna('S')
    df = df.drop(['SibSp','Parch','Name','Sex','Ticket','Cabin','Embarked'],axis=1)
    # One hot encoding for "Pclass"
    df = df.join(pd.get_dummies(df['Pclass'], prefix ='Class').astype(int))
    df = df.drop(['Pclass'], axis = 1)
    return df
train_df = load_clean_data('train.csv')
test_df = load_clean_data('test.csv')
train_df.describe()

,PassengerId,Survived,Age,Fare,Gender,Class_1,Class_2,Class_3
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,-0.132980,-0.229728,0.647587,0.242424,0.206510,0.551066
std,257.353842,0.486592,0.162746,0.272966,0.477990,0.428790,0.405028,0.497665
min,1.000000,0.000000,-0.494750,-0.500000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,-0.225000,-0.420896,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,-0.150000,-0.355458,1.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,-0.062500,-0.190000,1.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,0.500000,0.500000,1.000000,1.000000,1.000000,1.000000


In [10]:
x_data = train_df.drop(['PassengerId', "Survived"], axis = 1)
y_data = train_df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

In [11]:
classifier = learn.DNNClassifier(hidden_units = [10, 20, 10],
                                 n_classes=2)
classifier.fit(x = x_train, y = y_train, steps = 2000)

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.


DNNClassifier(hidden_units=[10, 20, 10], dropout=None, optimizer=None, feature_columns=None)

In [16]:
accuracy_score = classifier.evaluate(x = x_test, y = y_test)["accuracy"]
print('Accuracy: ', accuracy_score)

Accuracy:  0.759777


In [15]:
test_data = test_df.drop(['PassengerId'], axis = 1)
y_pred = classifier.predict(test_data)
submission = pd.DataFrame({ "PassengerId":test_df['PassengerId'],
                             "Survived":y_pred })
submission.to_csv('titanic_tf_learn.csv',index=False)